In [1]:
import socket
import cv2
import numpy as np
import pickle

In [9]:
# 创建 TCP 客户端套接字
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(("localhost", qtPort))  # 本地端口连接Qt程序

# 创建视频接收的 TCP 服务器套接字（接收远程视频流）
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((videoIP, videoPort))  # 远程设备连接到这个端口
server_socket.listen(1)
print("Waiting for video connection...")
video_socket, addr = server_socket.accept()
print(f"Connection established with {addr}")

# 接收视频流并进行处理
while True:
    # 接收数据
    data = b""
    while len(data) < 88888:
        packet = video_socket.recv(88888)
        if not packet:
            break
        data += packet

    # 解码图像
    img = np.frombuffer(data, dtype=np.uint8)
    frame = cv2.imdecode(img, cv2.IMREAD_COLOR)

    # 进行深度学习处理，假设使用 OpenCV 或深度学习框架进行处理
    # 假设识别结果是一个框，识别出物体并标记
    # 假设识别框位置和标签
    '''
    boxes = [(50, 50, 100, 100, "Object1"), (150, 150, 120, 120, "Object2")]
    for box in boxes:
        x, y, w, h, label = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # 绘制框
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    '''

    # 编码图像为 JPEG 格式
    _, img_encoded = cv2.imencode('.jpg', frame)
    img_bytes = img_encoded.tobytes()  # 转为字节数据

    # 发送编码后的图像到 Qt
    try:
        client_socket.sendall(img_bytes)
    except Exception as e:
        print(f"Error sending image to Qt: {e}")
        break

client_socket.close()
video_socket.close()
server_socket.close()


OSError: [Errno 9] Bad file descriptor